# Food Recipe Browser Project

By `652115013 Narongchai Rongthong`

In [4]:
import pandas as pd

# Load recipe data from Parquet (ignoring MongoDB for now)
recipes_df = pd.read_parquet('resource/recipes.parquet')
print(f"Loaded {len(recipes_df)} recipes.")

Loaded 522517 recipes.


In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "_Z9BSk2zcMuFD=-1LlAX"),
    ca_certs="~/http_ca.crt"
)

if es_client.ping():
    print("Connected to Elasticsearch")
else:
    print("Elasticsearch connection failed")

Connected to Elasticsearch


In [8]:
import json

index_name = "recipes"

# Delete the index if it already exists
es_client.indices.delete(index=index_name, ignore=[400,404])

# Create the index with a mapping that uses an English analyzer
mapping = {
    "settings": {
        "analysis": {
            "analyzer": {
                "default": {
                    "type": "english"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "recipe_id": {"type": "keyword"},
            "name": {"type": "text", "analyzer": "english"},
            "ingredients": {"type": "text", "analyzer": "english"},
            "instructions": {"type": "text", "analyzer": "english"},
            "full_text": {"type": "text", "analyzer": "english"},
            "image_url": {"type": "keyword"}
        }
    }
}

es_client.indices.create(index=index_name, body=mapping)
print(f"Created index: {index_name}")

# Index documents from recipes_df
import numpy as np

for idx, row in recipes_df.iterrows():
    recipe_id = str(row.get('RecipeId', idx))
    name = row.get('Name', '')
    ingredients = " ".join(row.get('RecipeIngredientParts', []))
    instructions = " ".join(row.get('RecipeInstructions', []))
    full_text = " ".join([name, ingredients, instructions])
    image_url = row.get('ImageUrl', '')  # Adjust if your data contains image URLs
    doc = {
        "recipe_id": recipe_id,
        "name": name,
        "ingredients": ingredients,
        "instructions": instructions,
        "full_text": full_text,
        "image_url": image_url
    }
    es_client.index(index=index_name, id=recipe_id, body=doc)

print("Indexed recipes into Elasticsearch.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_11280\412607391.py:6: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=index_name, ignore=[400,404])


Created index: recipes
Indexed recipes into Elasticsearch.


In [9]:
# --- Dummy User & In-Memory Data for Auth, Bookmarks, and Folders ---

# Dummy user database (for UC-001: Authentication)
users = {
    "user1": "password1",
    "user2": "password2"
}

# In-memory storage
sessions = {}  # token -> username
user_bookmarks = {}  # username -> list of {recipe_id, rating}
user_folders = {}   # username -> {folder_name: [recipe_ids]}

import uuid

def generate_token():
    return str(uuid.uuid4())

print("Initialized dummy user authentication and storage.")

Initialized dummy user authentication and storage.


In [14]:
# --- Flask API Endpoints ---
from flask import Flask, request, jsonify
import random
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Allows all origins (for development)

# UC-001: User Authentication
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get("username")
    password = data.get("password")
    if username in users and users[username] == password:
        token = generate_token()
        sessions[token] = username
        return jsonify({"message": "Login successful", "token": token})
    return jsonify({"message": "Invalid credentials"}), 401

@app.route('/logout', methods=['POST'])
def logout():
    token = request.headers.get("Authorization")
    if token in sessions:
        sessions.pop(token)
        return jsonify({"message": "Logout successful"})
    return jsonify({"message": "Invalid token"}), 401

# UC-002 & UC-003: Recipe Search Functionality & Display Results
@app.route('/search', methods=['GET'])
def search():
    token = request.headers.get("Authorization")
    if token not in sessions:
        return jsonify({"message": "Unauthorized"}), 401
    query = request.args.get("query", "")
    res = es_client.search(index=index_name, body={
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["name", "ingredients", "instructions", "full_text"]
            }
        }
    })
    hits = res["hits"]["hits"]
    results = [
        {
            "recipe_id": hit["_source"]["recipe_id"],
            "name": hit["_source"]["name"],
            "snippet": hit["_source"]["full_text"][:150],
            "image_url": hit["_source"].get("image_url", "")
        } for hit in hits
    ]
    return jsonify({"results": results})

# UC-004: Detailed Dish Information
@app.route('/recipe/<recipe_id>', methods=['GET'])
def recipe_detail(recipe_id):
    token = request.headers.get("Authorization")
    if token not in sessions:
        return jsonify({"message": "Unauthorized"}), 401
    res = es_client.get(index=index_name, id=recipe_id)
    return jsonify(res["_source"])

# UC-006: Bookmarking and Rating
@app.route('/bookmark', methods=['POST'])
def bookmark():
    token = request.headers.get("Authorization")
    if token not in sessions:
        return jsonify({"message": "Unauthorized"}), 401
    data = request.get_json()
    recipe_id = data.get("recipe_id")
    rating = data.get("rating")
    username = sessions[token]
    user_bookmarks.setdefault(username, []).append({"recipe_id": recipe_id, "rating": rating})
    return jsonify({"message": "Bookmarked successfully"})

# UC-005: Folder Management
@app.route('/folders', methods=['GET', 'POST'])
def folders():
    token = request.headers.get("Authorization")
    if token not in sessions:
        return jsonify({"message": "Unauthorized"}), 401
    username = sessions[token]
    if request.method == 'GET':
        return jsonify(user_folders.get(username, {}))
    elif request.method == 'POST':
        data = request.get_json()
        folder_name = data.get("folder_name")
        user_folders.setdefault(username, {})[folder_name] = []
        return jsonify({"message": f"Folder '{folder_name}' created"})

# UC-007: Personalized Recommendations (dummy implementation)
@app.route('/recommendations', methods=['GET'])
def recommendations():
    token = request.headers.get("Authorization")
    if token not in sessions:
        return jsonify({"message": "Unauthorized"}), 401
    # For demonstration, return 5 random recipes
    res = es_client.search(index=index_name, body={
        "query": {"match_all": {}},
        "size": 5
    })
    hits = res["hits"]["hits"]
    recs = [
        {
            "recipe_id": hit["_source"]["recipe_id"],
            "name": hit["_source"]["name"],
            "snippet": hit["_source"]["full_text"][:150],
            "image_url": hit["_source"].get("image_url", "")
        } for hit in hits
    ]
    return jsonify({"recommendations": recs})

# (UC-008: Suggestion List Generation would normally involve a learn-to-rank model. Here we leave it as a placeholder.)

print("Flask API endpoints defined.")

Flask API endpoints defined.


In [15]:
# Run the Flask app on port 5000
app.run(port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Mar/2025 09:34:43] "GET /login HTTP/1.1" 405 -
127.0.0.1 - - [02/Mar/2025 09:35:06] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2025 09:37:01] "GET /search?query=chicken HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2025 09:37:18] "GET /search?query=chicken%20rice HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2025 09:37:40] "GET /search?query=chicken%20rice HTTP/1.1" 200 -


### Testing Instructions

1. **Authentication:** Use a REST client (or cURL) to POST to `/login` with JSON payload, e.g.: 
   ```json
   {"username": "user1", "password": "password1"}
   ```
   You'll receive a token in the response. Use that token in the `Authorization` header for subsequent requests.

2. **Search:** GET `/search?query=chicken` with the header `Authorization: <token>` to retrieve matching recipes.

3. **Detailed View:** GET `/recipe/<recipe_id>` to fetch full details for a recipe.

4. **Bookmarking:** POST to `/bookmark` with JSON payload containing a `recipe_id` and an optional `rating`.

5. **Folder Management:** GET or POST to `/folders` to list or create folders.

6. **Recommendations:** GET `/recommendations` to retrieve a list of recommended recipes (dummy implementation).